In [1]:
import json
import requests

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    #response = requests.post(automat_url,json=message['message'])
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        return {}
    print(response.json())
    return response.json()

def strider(message):
    url = 'http://robokop.renci.org:5781/query'
    strider_answer = post(strider,url,message)
    numanswers = len(strider_answer['results'])
    if (numanswers == 0) or ((numanswers == 1) and (len(strider_answer['results'][0]['node_bindings'])==0)):
        print('no answers')
        return {}
    #Strider for some reason doesn't return the query graph
    strider_answer['query_graph'] = message['message']['query_graph']
    return strider_answer
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

def aragorn(message, coalesce_type):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

In [25]:
from neo4j import GraphDatabase

class neo():
    def __init__(self,uri,user,pw):
        if user == '':
            self.driver = GraphDatabase.driver(uri)
        else:
            self.driver = GraphDatabase.driver(uri, auth=(user,pw))

    def run_query(self,q):
        with self.driver.session() as session:
            res = session.read_transaction(rq,q)
        return res

def rq(tx,cypher):
    result = tx.run(cypher)
    #print(result)
    results = [ r for r in result ]
    return results


Here's how you call the ROBOKOP KG

In [8]:
n = neo('bolt://robokopkg.renci.org:7687','','')

In [26]:
x = n.run_query("MATCH (a:gene {id:'NCBIGene:2332'}) RETURN a")

Which phenotypes are associated with the most mendelian genes?

This query is mendelian diseases to phenotypes, where those phenotypes are associated with a gene that causes the disease.  Then we group by phenotype and count the number of genes.

Note that there's no way to do this in TRAPI...

Note, that might be wrong, what if gene and disease are sets...?

In [66]:
query = 'match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature)--(g:gene)-[:causes]->(x) return p.id, p.name, count(g) as n order by n desc'

In [67]:
r = n.run_query(query)

In [68]:
for rec in r:
    print (rec['p.id'], rec['p.name'], rec['n'])

MONDO:0005027 epilepsy 574
HP:0002187 Intellectual disability, profound 494
MONDO:0001071 intellectual disability 424
HP:0001263 Global developmental delay 414
HP:0001319 Neonatal hypotonia 392
HP:0004322 Short stature 338
HP:0002650 Scoliosis 223
MONDO:0000437 cerebellar ataxia 223
HP:0001508 Failure to thrive 220
HP:0002515 Waddling gait 186
HP:0000347 Micrognathia 181
MONDO:0007778 hypertelorism (disease) 181
MONDO:0004843 pathologic nystagmus 178
MONDO:0003432 strabismus 175
HP:0000467 Neck muscle weakness 170
MONDO:0004730 speech disorder 156
MONDO:0009047 cryptorchidism (disease) 153
MONDO:0000728 ptosis (disease) 150
MONDO:0020678 sensorineural hearing loss disorder 144
HP:0002240 Hepatomegaly 140
MONDO:0005129 cataract (disease) 139
HP:0000445 Wide nose 132
MONDO:0001565 abnormal retinal correspondence 129
MONDO:0003608 optic atrophy 125
HP:0005280 Depressed nasal bridge 125
HP:0000218 High palate 125
MONDO:0005365 hearing loss disorder 124
HP:0001260 Dysarthria 117
HP:0000494 

HP:0000963 Thin skin 13
MONDO:0002181 exostosis 13
MONDO:0007959 medulloblastoma 13
MONDO:0002321 sensory peripheral neuropathy 13
HP:0007256 Abnormal pyramidal sign 13
HP:0008180 Mildly elevated creatine kinase 13
MONDO:0001576 telangiectasis 13
HP:0003312 Abnormal form of the vertebral bodies 13
HP:0000413 Atresia of the external auditory canal 13
HP:0003429 CNS hypomyelination 13
HP:0002073 Progressive cerebellar ataxia 13
HP:0002705 High, narrow palate 13
HP:0001181 Adducted thumb 13
HP:0000455 Broad nasal tip 13
HP:0001805 Onychogryposis 13
HP:0003687 Centrally nucleated skeletal muscle fibers 13
HP:0001800 Hypoplastic toenails 13
MONDO:0006810 intracranial hypertension 13
MONDO:0008637 bifid uvula 13
MONDO:0005387 primary ovarian failure 13
HP:0001507 Growth abnormality 13
HP:0006716 Hereditary nonpolyposis colorectal carcinoma 13
HP:0002529 Neuronal loss in central nervous system 12
HP:0003306 Spinal rigidity 12
HP:0008065 Aplasia/Hypoplasia of the skin 12
HP:0001320 Cerebellar 

HP:0002312 Clumsiness 6
HP:0000696 Delayed eruption of permanent teeth 6
MONDO:0006496 palsy 6
MONDO:0001076 glucose intolerance 6
HP:0000641 Dysmetric saccades 6
HP:0000831 Insulin-resistant diabetes mellitus 6
HP:0010311 Aplasia/Hypoplasia of the breasts 6
HP:0001178 Ulnar claw 6
MONDO:0005340 alopecia areata 6
HP:0003390 Sensory axonal neuropathy 6
HP:0000176 Submucous cleft hard palate 6
HP:0002389 Cavum septum pellucidum 6
HP:0002990 Fibular aplasia 6
MONDO:0019706 lysosomal storage disease with skeletal involvement 6
HP:0008388 Abnormal toenail morphology 6
HP:0001992 Organic aciduria 6
HP:0002967 Cubitus valgus 6
HP:0005474 Decreased calvarial ossification 6
MONDO:0002251 hepatitis 6
HP:0007973 Retinal dysplasia 6
HP:0007495 Prematurely aged appearance 6
HP:0000402 Stenosis of the external auditory canal 6
MONDO:0022349 congenital absence of septum pellucidum 6
MONDO:0007886 uterine corpus leiomyoma 6
HP:0008586 Hypoplasia of the cochlea 6
HP:0005815 Supernumerary ribs 6
HP:0040

HP:0001592 Selective tooth agenesis 3
HP:0010047 Short 5th metacarpal 3
MONDO:0008797 anodontia 3
HP:0012721 Venous malformation 3
MONDO:0005151 endocrine system disease 3
HP:0000086 Ectopic kidney 3
HP:0004446 Stomatocytosis 3
MONDO:0002800 thrombophlebitis 3
HP:0007334 Bilateral tonic-clonic seizure with focal onset 3
HP:0003417 Coronal cleft vertebrae 3
HP:0001239 Wrist flexion contracture 3
HP:0005830 Flexion contracture of toe 3
MONDO:0004933 hypoplastic left heart syndrome 3
HP:0010554 Cutaneous finger syndactyly 3
HP:0001040 Multiple pterygia 3
MONDO:0002272 polyclonal hypergammaglobulinemia 3
HP:0009773 Symphalangism affecting the phalanges of the hand 3
HP:0100540 Palpebral edema 3
HP:0000196 Lower lip pit 3
HP:0030260 Microphallus 3
HP:0040082 Happy demeanor 3
HP:0008915 Childhood-onset truncal obesity 3
HP:0009126 Increased adipose tissue 3
HP:0002297 Red hair 3
MONDO:0016523 bronchogenic cyst (disease) 3
HP:0009721 Shagreen patch 3
HP:0001951 Episodic ammonia intoxication 3

HP:0005294 Arterial dissection 3
HP:0011065 Conical incisor 3
HP:0007892 Hypoplasia of the lacrimal punctum 3
HP:0009637 Absent proximal phalanx of thumb 3
HP:0009740 Aplasia of the parotid gland 3
HP:0000198 Absence of Stensen duct 3
MONDO:0004926 dacryocystitis 3
MONDO:0006044 nephrosclerosis 3
HP:0007656 Lacrimal gland aplasia 3
HP:0007900 Hypoplastic lacrimal duct 3
HP:0005707 Bilateral triphalangeal thumbs 3
HP:0009462 Radial deviation of the 3rd finger 3
HP:0008743 Coronal hypospadias 3
HP:0100258 Preaxial polydactyly 3
HP:0100583 Corneal perforation 3
HP:0007732 Lacrimal gland hypoplasia 3
MONDO:0001170 hemiplegia 3
HP:0000914 Shield chest 3
MONDO:0005349 otosclerosis (disease) 3
MONDO:0007639 fundus albipunctatus 3
HP:0008501 Median cleft lip and palate 3
MONDO:0007885 Legg-Calve-Perthes disease 2
HP:0003370 Flat capital femoral epiphysis 2
HP:0010259 Cone-shaped epiphyses of the middle phalanges of the hand 2
HP:0007905 Abnormal iris vasculature 2
HP:0100299 Muscle fiber inclu

...

HP:0000642 Red-green dyschromatopsia 2
HP:0002176 Spinal cord compression 2
HP:0004631 Decreased cervical spine flexion due to contractures of posterior cervical muscles 2
HP:0001934 Persistent bleeding after trauma 2
HP:0000022 Abnormality of male internal genitalia 2
HP:0012557 EEG with centrotemporal focal spike waves 2
HP:0007068 Inferior vermis hypoplasia 2
MONDO:0007754 HYPRPP 2
HP:0008609 Morphological abnormality of the middle ear 2
HP:0005465 Facial hyperostosis 2
HP:0000350 Small forehead 2
HP:0012548 Fatty replacement of skeletal muscle 2
HP:0009886 Trichorrhexis nodosa 2
MONDO:0005872 nervous system cancer 2
MONDO:0005401 colonic neoplasm 2
HP:0100641 Neoplasm of the adrenal cortex 2
MONDO:0008380 retinoblastoma 2
MONDO:0002350 familial nephrotic syndrome 2
MONDO:0007699 Hashimoto thyroiditis 2
HP:0006946 Recurrent meningitis 2
MONDO:0011908 juvenile myelomonocytic leukemia 2
HP:0005108 Abnormality of the intervertebral disk 2
HP:0002344 Progressive neurologic deterioration

HP:0008449 Progressive cervical vertebral spine fusion 1
HP:0011987 Ectopic ossification in muscle tissue 1
HP:0011988 Ectopic ossification in tendon tissue 1
HP:0010054 Abnormality of the first metatarsal bone 1
HP:0004629 Small cervical vertebral bodies 1
HP:0011989 Ectopic ossification in ligament tissue 1
MONDO:0002492 acute kidney failure 1
HP:0002752 Sparse bone trabeculae 1
HP:0031415 High serum calcitriol 1
HP:0003013 Bulging epiphyses 1
HP:0003029 Enlargement of the ankles 1
HP:0003020 Enlargement of the wrists 1
HP:0001195 Single umbilical artery 1
HP:0006879 Pontocerebellar atrophy 1
MONDO:0019145 hereditary thrombophilia due to congenital protein C deficiency 1
MONDO:0002304 protein S deficiency 1
HP:0009824 Upper limb undergrowth 1
HP:0001103 Abnormal macular morphology 1
HP:0006402 Distal shortening of limbs 1
HP:0006042 Y-shaped metacarpals 1
HP:0010112 Mesoaxial foot polydactyly 1
HP:0010564 Bifid epiglottis 1
HP:0005151 Preductal coarctation of the aorta 1
HP:0008706 D

HP:0008285 Transient hypophosphatemia 1
HP:0011786 Thyrotoxicosis with toxic single thyroid nodule 1
MONDO:0001252 Plummer disease 1
HP:0003329 Hair shafts flattened at irregular intervals and twisted through 180 degrees about their axes 1
HP:0002589 Gastrointestinal atresia 1
HP:0001273 Abnormal corpus callosum morphology 1
HP:0007766 Optic disc hypoplasia 1
HP:0001958 Nonketotic hypoglycemia 1
HP:0008305 Exercise-induced myoglobinuria 1
HP:0009045 Exercise-induced rhabdomyolysis 1
HP:0002406 Limb dysmetria 1
MONDO:0015237 arrhinia 1
MONDO:0007935 cystoid macular edema 1
HP:0030053 Stiff skin 1
MONDO:0002637 histiocytosis 1
MONDO:0019289 hyperpigmentation of the skin 1
HP:0007943 Congenital stapes ankylosis 1
HP:0009177 Proximal/middle symphalangism of 5th finger 1
HP:0004977 Bilateral radial aplasia 1
HP:0008952 Shoulder muscle hypoplasia 1
HP:0006507 Aplasia/hypoplasia of the humerus 1
HP:0100694 Tibial torsion 1
MONDO:0005849 milk allergic reaction 1
HP:0002970 Genu varum 1
HP:0003

HP:0012802 Broad jaw 1
HP:0200056 Macular scar 1
HP:0010892 Abnormal circulating branched chain amino acid concentration 1
HP:0007979 Gaze-evoked horizontal nystagmus 1
HP:0007944 Intermittent microsaccadic pursuits 1
HP:0002060 Abnormal cerebral morphology 1
HP:0011834 Moyamoya phenomenon 1
HP:0007402 Areas of hypopigmentation and hyperpigmentation that do not follow Blaschko lines 1
MONDO:0008431 slipped femoral capital epiphyses 1
HP:0000690 Agenesis of maxillary lateral incisor 1
HP:0040012 Chromosome breakage 1
HP:0005598 Facial telangiectasia in butterfly midface distribution 1
MONDO:0018904 primary membranoproliferative glomerulonephritis 1
HP:0004630 Anterior beaking of thoracic vertebrae 1
HP:0008436 Absent/hypoplastic coccyx 1
HP:0005453 Absent/hypoplastic paranasal sinuses 1
HP:0004558 Cervical platyspondyly 1
HP:0010471 Oligosacchariduria 1
HP:0005264 Abnormality of the gallbladder 1
HP:0100732 Pancreatic fibrosis 1
HP:0008450 Narrow vertebral interpedicular distance 1
HP:0

HP:0200138 Bilateral choanal atresia/stenosis 1
HP:0007308 Extrapyramidal dyskinesia 1
MONDO:0001152 amnestic disorder 1
HP:0007338 Hypermetric saccades 1
HP:0007603 Freckles in sun-exposed areas 1
MONDO:0005075 thyroid gland papillary carcinoma 1
HP:0000866 Euthyroid multinodular goiter 1
HP:0100619 Sertoli cell neoplasm 1
MONDO:0003669 testicular seminoma 1
HP:0007545 Congenital palmoplantar keratosis 1
HP:0007633 Bilateral microphthalmos 1
MONDO:0017198 osteopetrosis (disease) 1
HP:0005170 Complete heart block with broad QRS complexes 1
HP:0006203 Decreased movement range in interphalangeal joints 1
HP:0008116 Flexion limitation of toes 1
HP:0004961 Pulmonary artery sling 1
HP:0000795 Abnormality of the urethra 1
HP:0005357 Defective B cell differentiation 1
HP:0006254 Elevated alpha-fetoprotein 1
HP:0008669 Abnormal spermatogenesis 1
MONDO:0018908 non-Hodgkin lymphoma 1
MONDO:0004952 Hodgkins lymphoma 1
HP:0004911 Episodic metabolic acidosis 1
HP:0003244 Penile hypospadias 1
HP:000

In [69]:
query = 'match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature)--(g:gene)-[:causes]->(x) return p.id, p.name, count(x) as n order by n desc'

In [70]:
r = n.run_query(query)

In [ ]:
for rec in r:
    print (rec['p.id'], rec['p.name'], rec['n'])

OK, if we restrict the above query to genes/diseases related to epilepsy, what do we get?

In [43]:
query = 'match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature {id:"MONDO:0005027"})--(g:gene)-[:causes]->(x) return distinct (g)'

In [44]:
r = n.run_query(query)

In [45]:
for res in r:
    print(res['g']['name'], res['g']['id'])

FGFR1 NCBIGene:2260
SLC25A12 NCBIGene:8604
MLH3 NCBIGene:27030
EPCAM NCBIGene:4072
PMS2 NCBIGene:5395
FAN1 NCBIGene:22909
MSH2 NCBIGene:4436
MLH1 NCBIGene:4292
MSH6 NCBIGene:2956
ACADM NCBIGene:34
VPS13A NCBIGene:23230
GLI3 NCBIGene:2737
RAI1 NCBIGene:10743
MOCS1 NCBIGene:4337
WDR73 NCBIGene:84942
DGUOK NCBIGene:1716
SDHA NCBIGene:6389
SDHD NCBIGene:6392
TUBGCP4 NCBIGene:27229
NDUFA1 NCBIGene:4694
NDUFS6 NCBIGene:4726
NDUFAF4 NCBIGene:29078
FOXRED1 NCBIGene:55572
NDUFV1 NCBIGene:4723
MT-ND1 NCBIGene:4535
NDUFAF3 NCBIGene:25915
NDUFS4 NCBIGene:4724
NDUFS1 NCBIGene:4719
NDUFB9 NCBIGene:4715
NDUFB3 NCBIGene:4709
NDUFAF1 NCBIGene:51103
NDUFV2 NCBIGene:4729
NDUFS8 NCBIGene:4728
NDUFS3 NCBIGene:4722
NDUFS2 NCBIGene:4720
XPR1 NCBIGene:9213
PDGFRB NCBIGene:5159
PDGFB NCBIGene:5155
SLC20A2 NCBIGene:6575
ECM1 NCBIGene:1893
SUCLG1 NCBIGene:8802
SOX11 NCBIGene:6664
SMARCE1 NCBIGene:6605
ARID1A NCBIGene:8289
SMARCA4 NCBIGene:6597
SMARCB1 NCBIGene:6598
GRIA4 NCBIGene:2893
GNAO1 NCBIGene:2775
KCNMA1 

In [46]:
print(len(r))

393


Which are connected together?  Do an overlay on the set of genes, and count the number of connections each gene has to other genes.

Note that we can do the overlay part, but the counting would need to be outside TRAPI (not a big problem I think)

In [58]:
query = '''match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature {id:"MONDO:0005027"})--(g:gene)-[:causes]->(x)
with collect(id(g)) as nodes
CALL apoc.algo.cover(nodes) YIELD rel
RETURN  startNode(rel).id as source, startNode(rel).name as sourcename, rel, endNode(rel).id as target, endNode(rel).name as targetname'''

In [59]:
r = n.run_query(query)

In [60]:
from collections import defaultdict
conncounts = defaultdict(int)
for rec in r:
    conncounts[(rec['source'],rec['sourcename'])] += 1
    conncounts[(rec['target'],rec['targetname'])] += 1
inv = [ (v,k) for k,v in conncounts.items() ]
inv.sort()

In [61]:
inv[-10:]

[(13, ('NCBIGene:207', 'AKT1')),
 (13, ('NCBIGene:7531', 'YWHAE')),
 (14, ('NCBIGene:1738', 'DLD')),
 (14, ('NCBIGene:2332', 'FMR1')),
 (14, ('NCBIGene:8517', 'IKBKG')),
 (15, ('NCBIGene:4292', 'MLH1')),
 (15, ('NCBIGene:4726', 'NDUFS6')),
 (19, ('NCBIGene:3064', 'HTT')),
 (20, ('NCBIGene:5621', 'PRNP')),
 (30, ('NCBIGene:4722', 'NDUFS3'))]

Which genes are the ones interacting with NDUFS3?

In [62]:
nq = 'match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature {id:"MONDO:0005027"})--(g:gene)-[:causes]->(x), (g)-[:molecularly_interacts_with]-(gg:gene {id:"NCBIGene:4722"}) return distinct x.id, x.name, g.id, g.name order by x.id'
ndufsresults = n.run_query(nq)

In [63]:
for r in ndufsresults:
    print(r['x.name'],r['g.name'])

Huntington disease HTT
cytochrome-c oxidase deficiency disease SCO1
cytochrome-c oxidase deficiency disease COX6B1
maple syrup urine disease BCKDHA
maple syrup urine disease DLD
mitochondrial complex I deficiency NDUFB9
mitochondrial complex I deficiency NDUFB3
mitochondrial complex I deficiency NDUFS4
mitochondrial complex I deficiency NDUFAF4
mitochondrial complex I deficiency NDUFS6
mitochondrial complex I deficiency NDUFAF1
mitochondrial complex I deficiency MT-ND1
mitochondrial complex I deficiency NDUFV1
mitochondrial complex I deficiency NDUFS8
mitochondrial complex I deficiency NDUFAF3
mitochondrial complex I deficiency NDUFS1
mitochondrial complex I deficiency NDUFV2
mitochondrial complex I deficiency NDUFS2
Leigh syndrome COX15
MELAS syndrome MT-ND1
3-methylglutaconic aciduria with cataracts, neurologic involvement and neutropenia CLPB
3-methylglutaconic aciduria, type VIII HTRA2


Most of the connections are to other genes that are all part of a protein complex, and share a single disease (the disease is mendelain, but not monogenic).

It's not obvious, but most of the other diseases here are also mitochondria related.  Need to do some more queries and/or AC to see if that can be made obvious with our tools...

PRNP

In [65]:
prnpq = 'match (d:disease {id:"MONDO:0003847"})<-[:subclass_of]-(x:disease)-[:has_phenotype]->(p:disease_or_phenotypic_feature {id:"MONDO:0005027"})--(g:gene)-[:causes]->(x), (g)-[:molecularly_interacts_with]-(gg:gene {id:"NCBIGene:5621"}) return distinct x.id, x.name, g.id, g.name order by x.id'
prnpresults = n.run_query(prnpq)
for r in prnpresults:
    print(r['x.name'],r['g.name'])

dystonia 12 ATP1A3
Huntington disease HTT
linear nevus sebaceus syndrome HRAS
oculodentodigital dysplasia GJA1
neuronal ceroid lipofuscinosis 2 TPP1
cytochrome-c oxidase deficiency disease COX6B1
DOORS syndrome TBC1D24
mitochondrial complex I deficiency NDUFS6
isolated sulfite oxidase deficiency SUOX
X-linked epilepsy-learning disabilities-behavior disorders syndrome SYN1
X-linked complicated corpus callosum dysgenesis L1CAM
X-linked hydrocephalus with stenosis of the aqueduct of Sylvius L1CAM
syndromic X-linked intellectual disability Snyder type SMS
ethylmalonic encephalopathy ETHE1
Huntington disease-like 1 PRNP
epileptic encephalopathy, early infantile, 12 PLCB1
alternating hemiplegia of childhood 2 ATP1A3
sulfite oxidase deficiency due to molybdenum cofactor deficiency type C GPHN
microcephaly-corpus callosum hypoplasia-intellectual disability-facial dysmorphism syndrome PPP2R1A
hereditary hyperekplexia GPHN
epileptic encephalopathy, early infantile, 53 SYNJ1
epileptic encephalopa

Not as completely clear what these have to do with one another, but also haven't spent much time looking for answers.